In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import Recommender

In [2]:
production=True

In [3]:
data = pd.read_csv('data/retail_train.csv')
item_features = pd.read_csv('data/product.csv')
user_features = pd.read_csv('data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

In [4]:
train, val = Recommender.split_by_weeks(data, 0 if production else 4)
print(f"В тренировочном наборе {len(train)} записей, в валидационном наборе {len(val)}")

В тренировочном наборе 2396804 записей, в валидационном наборе 0


In [5]:
n_items_before = train['item_id'].nunique()

train = prefilter_items(train, item_features=item_features, take_n_popular=5000)

n_items_after = train['item_id'].nunique()
print(f'Число товаров уменьшено с {n_items_before} до {n_items_after}')
print(f"В тренировочном наборе {len(train)} записей")

Число товаров уменьшено с 89051 до 5000
В тренировочном наборе 1483107 записей


In [6]:
# вся модель в src.recommenders.Recommender
recommender = Recommender(train, user_features, item_features, n_lvl_2_train_weeks=12, n_lvl_2_train_candidates=20, n_lvl_2_predict_candidates=50)


Подготовка 2-х уровневой модели ...
Выделено 208972 из 1483107 записей для 12 последних недель.
Обучение: подготовка списка всех пользователей с кандидатами (20 кандидатов для каждого пользователя) ...
Обучение: подготовлено 49620 записей 2481 пользователей с кандидатами.
Обучение: добавление признаков ...
Обучение: добавление признаков завершено, число записей: 49587 ...
Число записей для пользователей из 12 последних недель: 45358 ...
Размер тренировочного набора: 63093
1.0    33598
0.0    29495
Name: target, dtype: int64
Категориальные признаки: ['department', 'brand', 'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product', 'age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc', 'hh_comp_desc', 'household_size_desc', 'kid_category_desc']
Тренируем модель 2го уровня ...


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Готовим рекомендации 2го уровня ...
Предсказания: подготовка списка всех пользователей с кандидатами (50 кандидатов для каждого пользователя) ...
Предсказания: подготовлено 124050 записей 2481 пользователей с кандидатами.
Предсказания: добавление признаков ...
Предсказания: добавление признаков завершено, число записей: 123718 ...
Подготовка 2-х уровневой модели завершена


In [7]:
if production:
    test_users = pd.read_csv('data/test_users.csv')
    test_users.columns=['UserId']
    test_users['Predicted']=test_users['UserId'].apply(lambda u: ' '.join(map(str, recommender.get_recommendations('lvl_2', u, 5))))
    test_users.to_csv('submission.csv', index=False)

Нет рекомендации для user_id=62 для level=lvl_2, возвращаем 5 самых популярных
Нет рекомендации для user_id=75 для level=lvl_2, возвращаем 5 самых популярных
Нет рекомендации для user_id=650 для level=lvl_2, возвращаем 5 самых популярных
Нет рекомендации для user_id=954 для level=lvl_2, возвращаем 5 самых популярных
Нет рекомендации для user_id=1043 для level=lvl_2, возвращаем 5 самых популярных
Нет рекомендации для user_id=1176 для level=lvl_2, возвращаем 5 самых популярных
Нет рекомендации для user_id=2120 для level=lvl_2, возвращаем 5 самых популярных
Нет рекомендации для user_id=2325 для level=lvl_2, возвращаем 5 самых популярных
Нет рекомендации для user_id=2364 для level=lvl_2, возвращаем 5 самых популярных


In [8]:
if not production:
    result_val = val.groupby('user_id')['item_id'].unique().reset_index().copy()
    result_val.columns=['user_id', 'actual']

    for model_name in ['top', 'own', 'lvl_2']:
        result_val[model_name] = result_val['user_id'].apply(lambda u: recommender.get_recommendations(model_name, u, 5))
        result_val['p@k_' + model_name] = result_val.apply(lambda row: precision_at_k(row[model_name], row['actual']), axis=1)

    for model_name in ['top', 'own', 'lvl_2']:
        mean_precision_at_k = result_val['p@k_' + model_name].mean()
        print(f'Model: {model_name}, Mean precision@k = {mean_precision_at_k}')